In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
import pytorch_lightning as pl

from torch_geometric.data import DataLoader
import pytorch_lightning as pl
import os.path as osp

from tqdm import tqdm

from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

In [2]:
%cd ..

/nfs/homedirs/fuchsgru/MastersThesis


In [10]:
import configuration
import model_registry
import data.build
import data.constants as dconst
from data.util import data_get_num_attributes, data_get_num_classes
from model.semi_supervised_node_classification import SemiSupervisedNodeClassification, Ensemble
from evaluation.util import get_data_loader, run_model_on_datasets, separate_distributions
import evaluation.callbacks as ecbs
from metrics import expected_calibration_error
from plot.calibration import plot_calibration
from data.util import label_binarize
from model.density import get_density_model
import seed
from train import train_model
from model.build import make_model
import model.constants as mconst

In [4]:
import logging as l

l.basicConfig(level=l.INFO)

In [5]:
dataset = dconst.CORA_FULL

spectral_norm_conf = {'model' : {
        'use_spectral_norm' : True,
        'residual' : True,
        'weight_scale' : 200.0,
    },}

config = configuration.ExperimentConfiguration(
    data={
        'dataset' : dataset,
        'setting' : dconst.HYBRID,
        'ood_type' : dconst.PERTURBATION,
    },
    **(spectral_norm_conf),
    run = {
        'use_default_configuration' : True,
    },
)
config.model.use_bias = True
if config.run.use_default_configuration:
    configuration.update_with_default_configuration(config)
    
print(config.run.use_default_configuration)

INFO:root:Set configuration value data.base_labels to default ['Artificial_Intelligence/Machine_Learning/Case-Based', 'Artificial_Intelligence/Machine_Learning/Theory', 'Artificial_Intelligence/Machine_Learning/Genetic_Algorithms', 'Artificial_Intelligence/Machine_Learning/Probabilistic_Methods', 'Artificial_Intelligence/Machine_Learning/Neural_Networks', 'Artificial_Intelligence/Machine_Learning/Rule_Learning', 'Artificial_Intelligence/Machine_Learning/Reinforcement_Learning', 'Operating_Systems/Distributed', 'Operating_Systems/Memory_Management', 'Operating_Systems/Realtime', 'Operating_Systems/Fault_Tolerance']
INFO:root:Set configuration value data.corpus_labels to default ['Artificial_Intelligence/Machine_Learning/Case-Based', 'Artificial_Intelligence/Machine_Learning/Theory', 'Artificial_Intelligence/Machine_Learning/Genetic_Algorithms', 'Artificial_Intelligence/Machine_Learning/Probabilistic_Methods', 'Artificial_Intelligence/Machine_Learning/Neural_Networks', 'Artificial_Intell

True


In [12]:
data_split_seed = seed.data_split_seeds()[config.run.split_idx]
model_seed_generator = iter(seed.SeedIterator(seed.model_seeds()[config.run.initialization_idx]))
model_seed = next(model_seed_generator)
config.registry.model_seed = model_seed
config.registry.split_seed = data_split_seed
config.training.early_stopping.patience = 50
config.training.early_stopping.min_delta = 0.01


config.model.reconstruction.loss_weight = 1.0
config.model.reconstruction.reconstruction_type = mconst.ENERGY

In [19]:
config.run.use_pretrained_model

True

In [23]:
config.run.use_pretrained_model = True

if not torch.cuda.is_available():
    config.training.gpus = 0

In [24]:
data_dict, fixed_vertices = data.build.load_data_from_configuration(config.data, data_split_seed)

INFO:root:Found precomputed dataset split at /nfs/students/fuchsgru/dataset_registry/2361285485.pt


In [25]:
from torch_geometric.data import DataLoader
data_loaders = {
    name : DataLoader(data, batch_size=1, shuffle=False) for name, data in data_dict.items()
}

/nfs/homedirs/fuchsgru/miniconda3/lib/python3.9/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [26]:
import model_registry
registry = model_registry.ModelRegistry()

In [27]:
registry[config]

'/nfs/students/fuchsgru/model_registry/3202218202.ckpt'

In [28]:
run_artifact_dir = 'test'

In [29]:
model = make_model(config, data_get_num_attributes(data_dict[dconst.TRAIN][0]), 
data_get_num_classes(data_dict[dconst.TRAIN][0])).eval()
model = train_model(model, config, run_artifact_dir, data_loaders, logger=None).eval()
model.clear_and_disable_cache()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
INFO:root:Found pre-trained model at /nfs/students/fuchsgru/model_registry/3202218202.ckpt
INFO:root:Loading model for evaluation from /nfs/students/fuchsgru/model_registry/3202218202.ckpt.
INFO:root:<class 'model.nn.GCNConv'> disabled cache.
INFO:root:<class 'model.nn.GCNConv'> disabled cache.


In [30]:
with torch.no_grad():
    data_test = data_dict[dconst.TEST][0]
    pred_test = model(data_test)
    acc_test = (pred_test.get_predictions().argmax(dim=1) == data_test.y)[data_test.mask].float().mean()
    

In [31]:
acc_test

tensor(0.7844)

In [33]:
W = model.get_weights()['backbone.conv0.lin.weight'].detach().cpu().numpy()

In [34]:
U, S, V = np.linalg.svd(W, full_matrices=False)

In [39]:
S / S.max()

array([1.        , 0.34644157, 0.21357606, 0.18221658, 0.17845054,
       0.17358452, 0.17115714, 0.16369098, 0.16322312, 0.16211107,
       0.16065218, 0.1579014 , 0.15761709, 0.15563282, 0.15385689,
       0.15299924, 0.1520852 , 0.15121146, 0.15073255, 0.1487227 ,
       0.1480999 , 0.14714998, 0.14621434, 0.14535412, 0.14484566,
       0.14467412, 0.14359272, 0.14320074, 0.14250769, 0.14214145,
       0.14086385, 0.14054221, 0.1399537 , 0.13944867, 0.13895313,
       0.13866158, 0.13808402, 0.1368391 , 0.13634303, 0.13609466,
       0.1353066 , 0.13461395, 0.13378777, 0.13345309, 0.13289294,
       0.13199455, 0.13152133, 0.13077731, 0.13032141, 0.13027649,
       0.13000262, 0.12891391, 0.12799823, 0.12694843, 0.12627207,
       0.125105  , 0.12487053, 0.12399704, 0.12364048, 0.12215623,
       0.12190119, 0.1207462 , 0.11943238, 0.11732855], dtype=float32)

In [41]:
np.linalg.matrix_rank(W)

64